# S&P 500 Historical Composition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils as u
import cvxpy as cp

data = pd.read_csv('./data/S&P 500 Historical Composition.csv')
data.head()

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


In [2]:
data['tickers'] = data['tickers'].apply(lambda x: x.split(','))
data.index = pd.to_datetime(data['date'])
data = data.drop('date', axis=1)
data.head()

,tickers
date,
1996-01-02,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-03,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-04,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-10,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-11,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."


In [3]:
all_stocks = set()
for stocks in data['tickers'].loc[data.index > '2016-01-01']:
    for stock in stocks:
        all_stocks.add(stock)

print('Number of unique stocks:', len(all_stocks))

Number of unique stocks: 706


In [4]:
curr_stocks = set()
for stocks in data['tickers'].loc[data.index > '2024-11-01']:
    for stock in stocks:
        curr_stocks.add(stock)

stock_data = u.get_stock_data()
cols = [stock for stock in curr_stocks if stock in stock_data.columns]
stock_data = stock_data[cols].loc[stock_data.index > '2019-01-01']
description = stock_data.describe().T
good_columns = description.loc[description['count'] == 1507].index
stock_data = stock_data[good_columns]

c:\Users\ttfee\OneDrive\Desktop\Stock Forecasting\utils.py:52: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.index = df.index.to_period('D')


In [26]:
import utils as u
import cvxpy as cp

Q = u.quadratic_form(stock_data)
r = u.returns(stock_data)

p = cp.Variable(Q.shape[0])
constraints = [
    cp.sum(p) == 1,
    p >= 0,
    p <= 0.1
]

# U(p; beta) = returns - beta * trend-weighted risk
betas = [10**i for i in np.linspace(-3, -1, 20)]
for beta in betas:
    function = r.transpose() @ p - beta * cp.quad_form(p, Q, assume_PSD=True)
    objective = cp.Maximize(function)
    problem = cp.Problem(objective, constraints)
    problem.solve()

    series = pd.Series(p.value, index=stock_data.columns)
    print(beta)
    series.sort_values(ascending=False, inplace=True)
    display(series.loc[series > 0.01])

0.001


TDG     0.1
CHTR    0.1
EQIX    0.1
BLK     0.1
REGN    0.1
BKNG    0.1
ORLY    0.1
MTD     0.1
NVR     0.1
AZO     0.1
dtype: float64

0.0012742749857031334


CHTR    0.100000
EQIX    0.100000
BLK     0.100000
NVR     0.100000
BKNG    0.100000
TMO     0.100000
MTD     0.100000
REGN    0.100000
ORLY    0.100000
AZO     0.099999
dtype: float64

0.001623776739188721


...

RTX     0.100003
BA      0.100002
BIIB    0.100002
FTV     0.100002
HUM     0.100002
TFX     0.100001
NOC     0.100001
MKTX    0.100001
CHTR    0.100000
MTD     0.032677
BKNG    0.029835
EQIX    0.024913
AZO     0.011340
dtype: float64

0.07847599703514611


FTV     0.100000
MKTX    0.100000
NOC     0.100000
CHTR    0.100000
HUM     0.100000
TFX     0.100000
RTX     0.100000
BIIB    0.100000
BA      0.100000
BDX     0.037797
BKNG    0.023886
AZO     0.018561
MTD     0.014755
dtype: float64

0.1


MKTX    0.100000
BIIB    0.100000
TFX     0.100000
FTV     0.100000
BA      0.100000
HUM     0.100000
RTX     0.100000
CHTR    0.100000
NOC     0.100000
BDX     0.055951
AZO     0.023288
BKNG    0.018353
dtype: float64